In [1]:
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

In [2]:
import os

datasets_path = 'datasets'
if not os.path.exists(datasets_path):
    os.makedirs(datasets_path)

small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')
     

In [4]:
import urllib

small_f = urllib.urlretrieve (small_dataset_url, small_dataset_path)


AttributeError: ignored

In [5]:
import urllib.request

small_f = urllib.request.urlretrieve(small_dataset_url, small_dataset_path)

In [7]:
import zipfile

with zipfile.ZipFile(small_dataset_path, "r") as z:
    z.extractall(datasets_path)



In [11]:
!pip install pyspark

from pyspark import SparkContext

sc = SparkContext("local[*]", "MovieRecommendation")

small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')

small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 20.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=089d375fe4feb5ab183b95e2d6fb79c057ebd3ef4e70977c32d9dd74045494a7
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [12]:
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [13]:
small_ratings_data.take(3)

[('1', '1', '4.0'), ('1', '3', '4.0'), ('1', '6', '4.0')]

In [14]:
small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')

small_movies_raw_data = sc.textFile(small_movies_file)
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
small_movies_data.take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

In [26]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import Row

In [33]:
# Load and parse the data
data = sc.textFile(small_ratings_file)
header = data.first()
data = data.filter(lambda row: row != header)
ratings = data.map(lambda l: l.split(',')).map(lambda l: Row(userId=int(l[0]), movieId=int(l[1]), rating=float(l[2]), timestamp=int(l[3])))
ratings_df = spark.createDataFrame(ratings)

In [32]:
import os

small_ratings_file = "datasets/ml-latest-small/ratings.csv"
if os.path.exists(small_ratings_file):
    print("File exists")
else:
    print("File does not exist")

File exists


In [34]:
# Define the hyperparameters
ranks = [5, 10, 20]
lambdas = [0.01]
num_iterations = [10]

In [43]:
# Iterate over the hyperparameters
for rank in ranks:
  for lambda_val in lambdas:
    for num_iter in num_iterations:
      
# Build the recommendation model using ALS on the training data
      als = ALS(maxIter=num_iter, regParam=lambda_val, rank=rank, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
      model = als.fit(ratings_df)

In [44]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [45]:
# Print the results
print("Rank: ", rank, ", lambda: ", lambda_val, ", numIter: ", num_iter, ", RMSE: ", rmse)

Rank:  20 , lambda:  0.01 , numIter:  10 , RMSE:  0.3603422364405746
